<a href="https://colab.research.google.com/github/partho1999/Reinforcement-Learning/blob/main/OpenAI_Reinforcement_Learning_with_Custom_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Install Dependencies

In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

1. Test Random Environment with OpenAI Gym

In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [ ]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 38 + random.randint(-3,3)
        # Reset shower time
        self.shower_length = 60 
        return self.state
    

In [ ]:
env = ShowerEnv()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [ ]:
env.observation_space.sample()

array([46.20065], dtype=float32)

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-16
Episode:2 Score:16
Episode:3 Score:-60
Episode:4 Score:-24
Episode:5 Score:-26
Episode:6 Score:-20
Episode:7 Score:-58
Episode:8 Score:-26
Episode:9 Score:-16
Episode:10 Score:-18


2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
actions

3

In [ ]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model
    del model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn
    

In [ ]:
del dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 7:50 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 75s 8ms/step - reward: -0.7656
166 episodes - episode_reward: -45.952 [-60.000, 14.000] - loss: 2.281 - mae: 10.167 - mean_q: -11.449

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 67s 7ms/step - reward: -0.5866
167 episodes - episode_reward: -35.305 [-60.000, 38.000] - loss: 2.561 - mae: 11.518 - mean_q: -16.521

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: -0.5684
167 episodes - episode_reward: -34.024 [-60.000, 32.000] - loss: 3.169 - mae: 13.363 - mean_q: -19.334

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: -0.4790
166 episodes - episode_reward: -28.687 [-60.000, 48.000] - loss: 3.447 - mae: 14.187 - mean_q: -20.567

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 69s 7ms/step - reward: -0.4476
done, took 349.595 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: -54.000, steps: 60
Episode 3: reward: -56.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: -58.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -58.000, steps: 60
Episode 11: reward: -60.000, steps: 60
Episode 12: reward: -54.000, steps: 60
Episode 13: reward: -58.000, steps: 60
Episode 14: reward: -56.000, steps: 60
Episode 15: reward: -56.000, steps: 60
Episode 16: reward: -60.000, steps: 60
Episode 17: reward: -54.000, steps: 60
Episode 18: reward: -58.000, steps: 60
Episode 19: reward: -60.000, steps: 60
Episode 20: reward: -54.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -54.000, steps: 60
Episode 23: reward: -60.000, steps: 60
Episode 24: reward: -54.000, steps: 60
Episode 25: reward: -60.000, steps: 60
Episo

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: -58.000, steps: 60
Episode 2: reward: -54.000, steps: 60
Episode 3: reward: -56.000, steps: 60
Episode 4: reward: -56.000, steps: 60
Episode 5: reward: -60.000, steps: 60
Episode 6: reward: -58.000, steps: 60
Episode 7: reward: -58.000, steps: 60
Episode 8: reward: -54.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -54.000, steps: 60
Episode 12: reward: -60.000, steps: 60
Episode 13: reward: -58.000, steps: 60
Episode 14: reward: -54.000, steps: 60
Episode 15: reward: -56.000, steps: 60


In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)